<a href="https://colab.research.google.com/github/chloevan/gcp_tutorial/blob/master/02_basics/Ch21_conditional_expressions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
title: "Ch21 Conditional Expressions"
date: 2020-05-28T12:30:30+09:00
tags:
  - "구글 클라우드"
  - "Google Cloud"
  - "빅쿼리"
  - "BigQuery"
  - "빅쿼리 파이썬"
  - "빅쿼리 기본"
  - "Conditional Expressions"
  - "BigQuery with Python"
categories:
  - "구글 클라우드"
  - "Google Cloud"
  - "빅쿼리"
  - "BigQuery"
  - "Python"
menu: 
  gcp:
    name: Ch21 Conditional Expressions
---

## I. 구글 클라우드 설정

본격적인 빅쿼리 실습에 앞서서, Python과 연동하는 예제를 준비하였다. 빅쿼리 시작에 앞서서 선행적으로 클라우드 사용을 해야 한다. 

1. 만약 GCP 프로젝트가 없다면, 계정을 연동한다. Go to [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager)
2. 그리고, 비용결제를 위한 카드를 등록한다. [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) 
3. 마지막으로 BigQuery API를 사용해야 하기 때문에 빅쿼리 API 사용허가를 내준다.[Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery)

위 API를 이용하지 않으면 `Python` 또는 `R`과 연동해서 사용할 수는 없다. 자주 쓰는것이 아니라면 비용은 거의 발생하지 않으니 염려하지 않아도 된다. 비용관리에 대한 자세한 내용은 [BigQuery 권장사항: 비용 관리](https://cloud.google.com/bigquery/docs/best-practices-costs?hl=ko)에서 확인하기를 바란다.  


## II. 사용자 계정 인증
구글 코랩을 사용해서 인증 절차를 밟도록 한다. 아래 소스코드는 변경시키지 않는다. 아래 절차대로 진행하면 된다. `Gmail` 인증 절차와 비슷하다.

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## III. Conditional Expressions
- `Booleans`가 유용하다는 것은 WHERE 조항에만 있는 것이 아니다.
- `SELECT`의 조건부 식을 사용하여 많은 쿼리를 단순화할 수 있다. 
- 예를 들어, 원하는 마크업과 해당 항목에 해당하는 세율을 기준으로 카탈로그에 있는 각 항목의 판매 가격을 계산해야 한다고 가정해 보자. 
- 카탈로그에 필요한 정보 중 일부가 누락된 경우 기본 마크업 또는 기본 세율을 귀속시킬 수 있다. 
- `IF` 기능을 사용하여 다음을 달성할 수 있다.

In [6]:
from google.cloud import bigquery
from tabulate import tabulate
import pandas as pd

project_id = 'bigquerytutorial-274406'
client = bigquery.Client(project=project_id)

temp = client.query('''
  WITH catalog AS (
    SELECT 30.0 AS costPrice, 0.15 AS markup, 0.1 AS taxRate
    UNION ALL SELECT NULL, 0.21, 0.15
    UNION ALL SELECT 30.0, NULL, 0.09
    UNION ALL SELECT 30.0, 0.30, NULL
    UNION ALL SELECT 30.0, NULL, NULL
  )
  SELECT 
    *, ROUND(
      costPrice * 
      IF(markup IS NULL, 1.05, 1+markup) * 
      IF(taxRate IS NULL, 1.10, 1+taxRate)
    , 2) AS salesPrice
  FROM catalog
  ''').to_dataframe()

print(temp)

   costPrice  markup  taxRate  salesPrice
0       30.0    0.15     0.10       37.95
1        NaN    0.21     0.15         NaN
2       30.0     NaN     0.09       34.34
3       30.0    0.30      NaN       42.90
4       30.0     NaN      NaN       34.65


- 이렇게 하면 비용을 알 수 없는 항목을 제외한 모든 품목에 대해 유효한 판매 가격을 산출할 수 있다.
- `IF` 함수의 작동 방식은 첫 번째 매개변수가 평가 조건이라는 것이다. 
- 첫번째 파라미터의 조건이 참이면 두 번째 파라미터가 사용되거나 또는 세 번째 파라미터가 사용된다. 
- 이 기능은 `SELECT` 에서 발생하기 때문에 행별로 수행된다.

## IV. Reference
Lakshmanan, V., & Tigani, J. (2020). Google BigQuery: the definitive guide: data warehousing, analytics, and machine learning at scale. Beijing: OReilly.